
# ΔNFR gradient fields

## Objectives
- Explain how ΔNFR gradients encode spatial reorganisation pressure across a lattice of nodes.
- Capture the deterministic hook that Phase-2 controllers require to seed reproducible ΔNFR fields before invoking adaptive solvers.
- Produce gradient telemetry (∇x, ∇y, magnitude) that the CI smoke tests can compare verbatim.

## Phase-2 dependencies
- [Phase-2 integration notes](../fase2_integration.md) — the gradient ledger feeds the multi-scale remesh routine defined there.
- :mod:`tnfr.dynamics.dnfr` — Phase-2 reuses the same ΔNFR aggregation hooks once the parallel workers are enabled.
- :mod:`tnfr.metrics.coherence` — gradient-aware coherence caches expect the ΔNFR field to be written with the same aliases showcased here.

## Theoretical exposition
ΔNFR gradients measure how rapidly the reorganisation operator changes across neighbouring nodes. In canonical TNFR form, each glyph write stores a scalar ΔNFR in the node; the gradient field arises by comparing these scalars along orthogonal directions. A deterministic field lets us validate that the nodal equation integrates a consistent ΔNFR distribution before higher-order remeshers refine the lattice. The primer therefore constructs a small 2×2 grid, scripts ΔNFR values with a linear trend, and derives the gradient vectors analytically across the canonical emission→reception→coherence→resonance→transition pipeline.

## Deterministic smoke check
The code below assembles the grid, installs the scripted field, fires the canonical segment, and records the gradient map after each glyph. Any drift in alias resolution, lattice wiring, or ΔNFR bookkeeping will surface as a mismatch in the reported gradient magnitudes.


In [ ]:

from math import hypot
from typing import Dict, List, Tuple

from tnfr.constants import DNFR_PRIMARY, EPI_PRIMARY, VF_PRIMARY
from tnfr.dynamics import set_delta_nfr_hook
from tnfr.structural import Coherence, Emission, Reception, Resonance, Transition, create_nfr, run_sequence
from tnfr.types import TNFRGraph

Grid = Dict[Tuple[int, int], str]

def build_grid() -> tuple[Grid, str, TNFRGraph]:
    grid: Grid = {}
    anchor: str | None = None
    G: TNFRGraph | None = None
    for x in range(2):
        for y in range(2):
            name = f"cell-{x}-{y}"
            params = dict(epi=0.4 + 0.05 * x + 0.03 * y, vf=1.0 + 0.02 * x, theta=0.0)
            if G is None:
                G, node = create_nfr(name, **params)
                anchor = node
            else:
                _, node = create_nfr(name, graph=G, **params)
            grid[(x, y)] = node
            G.nodes[node]["pos"] = (x, y)
            if x > 0:
                G.add_edge(node, grid[(x - 1, y)])
            if y > 0:
                G.add_edge(node, grid[(x, y - 1)])
    assert G is not None and anchor is not None
    return grid, anchor, G


def gradient_map(graph: TNFRGraph, grid: Grid) -> dict[str, dict[str, float]]:
    def dnfr_at(coord: Tuple[int, int]) -> float:
        node = grid.get(coord)
        if node is None:
            return 0.0
        return float(graph.nodes[node].get(DNFR_PRIMARY, 0.0))

    grads: dict[str, dict[str, float]] = {}
    for (x, y), node in grid.items():
        dnfr = dnfr_at((x, y))
        right = dnfr_at((x + 1, y)) if (x + 1, y) in grid else dnfr
        left = dnfr_at((x - 1, y)) if (x - 1, y) in grid else dnfr
        up = dnfr_at((x, y + 1)) if (x, y + 1) in grid else dnfr
        down = dnfr_at((x, y - 1)) if (x, y - 1) in grid else dnfr
        grad_x = 0.5 * (right - left)
        grad_y = 0.5 * (up - down)
        grads[f"cell-{x}-{y}"] = {
            "ΔNFR": round(dnfr, 6),
            "∇x": round(grad_x, 6),
            "∇y": round(grad_y, 6),
            "|∇|": round(hypot(grad_x, grad_y), 6),
        }
    return grads


grid, anchor, G = build_grid()
initial_gradients = gradient_map(G, grid)
step_scalars = iter([1.0, 0.9, 0.8, 0.7, 0.6])
trace: List[dict[str, dict[str, float]]] = []


def scripted_gradient(graph: TNFRGraph, grid: Grid, scale: float) -> None:
    for (x, y), node in grid.items():
        data = graph.nodes[node]
        base = (0.02 * x - 0.015 * y) * scale
        vf = float(data[VF_PRIMARY])
        epi = float(data[EPI_PRIMARY])
        data[DNFR_PRIMARY] = base
        data[EPI_PRIMARY] = epi + vf * base
        data[VF_PRIMARY] = vf + 0.1 * base


def apply_script(graph: TNFRGraph) -> None:
    scale = next(step_scalars, 0.5)
    scripted_gradient(graph, grid, scale)
    trace.append(gradient_map(graph, grid))

set_delta_nfr_hook(G, apply_script, note="ΔNFR gradient field smoke")
run_sequence(G, anchor, [Emission(), Reception(), Coherence(), Resonance(), Transition()])

final_gradients = gradient_map(G, grid)

{
    "initial": initial_gradients,
    "final": final_gradients,
    "per_step": trace,
}
